# HR Analytics: Understanding Employee Attrition
Analyzing trends and patterns in employee attrition using HR analytics data from Kaggle. The data is on Atlas Lab employees.

## Objectives
- Understand the distribution of employee attributes (age, department, role, etc.)
- Explore patterns in employee attrition
- Analyze job and performance satisfaction metrics
- Identify factors correlated with attrition

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, normaltest, probplot

## Load and Merge Data

In [ ]:
# Load Data
df_employee = pd.read_csv('Employee.csv')
df_performance = pd.read_csv('PerformanceRating.csv')

# Merge Data
df_combined = pd.merge(df_employee, df_performance, on='EmployeeID')
print(df_combined.shape)
df_combined.head()

## Data Cleaning
- dropping unnecessary columns 
- fixing column types
- mapping binary and ordinal values

In [ ]:
# Missing values
print(df_combined.isnull().sum())

# Map binary values
df_combined['Attrition'] = df_combined['Attrition'].map({'Yes':1, 'No': 0})
df_combined['OverTime'] = df_combined['OverTime'].map({'Yes': 1, 'No':0})


# Convert ordinal values
from pandas.api.types import CategoricalDtype
levels = CategoricalDtype(
    categories=[1,2,3,4,5],
    ordered=True
)

cols_to_convert = [
    'EnvironmentSatisfaction',
    'JobSatisfaction',
    'RelationshipSatisfaction',
    'WorkLifeBalance',
    'SelfRating',
    'ManagerRating',
    'Education',
]
for col in cols_to_convert:
    df_combined[col] = df_combined[col].astype(levels)

# convert remaining categorical columns
categorical_cols = ['BusinessTravel', 'Department', 'State', 'Ethnicity', 'EducationField', 'JobRole', 
                    'MaritalStatus', 'StockOptionLevel', 'TrainingOpportunitiesWithinYear',
                    'TrainingOpportunitiesTaken'
                    ]
for col in categorical_cols:
    df_combined[col] = df_combined[col].astype('category')

# Date columns
df_combined['HireDate'] = pd.to_datetime(df_combined['HireDate'])
df_combined['ReviewDate'] = pd.to_datetime(df_combined['ReviewDate'])

# Fixing 'Marketing' typo
df_combined['EducationField'] = df_combined['EducationField'].str.strip().str.title()

# Merge Marketing variants
df_combined['EducationField'] = df_combined['EducationField'].replace({
    'Marketing ': 'Marketing',
    'Marketting': 'Marketing' 
})
# Drop unused columns
drop_columns = ['FirstName', 'LastName', 'PerformanceID']

df_combined = df_combined.drop(drop_columns, axis=1)

# Save cleaned data
df_combined.to_csv('cleaned_employee_data.csv', index=False)


## 1. Exploratory Data Analysis - Overview of Dataset

In [ ]:
# Set visual style
sns.set(style='whitegrid', palette='hls')
plt.rcParams['figure.figsize'] = (10, 6)

# Load the dataset
df = df_combined.copy()

print(df.shape)
df.info()
df.describe()

## Function 1: ql_stats
- a function to handle qualitative columns
- runs summary (counts & percentages) for the column parameter
- displays how many unique responses a category has, the most occurring response, and how many employees chose that response.
- plots countplot for visualization.

In [ ]:
def ql_stats(df, col):
    """
    Prints the summary and percentage of each category 
    in the specified column w/count plots.

    Parameters: 
    df (DataFrame): The DataFrame containing the data.
    col (str): The name of the column to summarize.
    """
    print(f"\n--- Categorical Summary: {col} ---")
    counts = df[col].value_counts(dropna=False)
    percentages = df[col].value_counts(normalize=True, dropna=False) * 100

    summary = pd.DataFrame({
        'Count': counts,
        'Percentages': percentages.round(2)
    })
    print(summary)
    print(f"Unique categories: {df[col].nunique(dropna=False)}")
    print(f"Most frequent: {df[col].mode()[0]}")

    plt.figure(figsize=(12, 6))
    sns.countplot(x=col, data=df)

    if col in ['Ethnicity', 'EducationField', 'JobRole']:    
        # rotate to make space for x labels
        plt.xticks(rotation=45, ha='right')
        plt.subplots_adjust(bottom=0.3)
    else:
        plt.tight_layout()
    
    # Save plot as JPG
    plt.title(f'Distribution of {col}')
    filename = f'Distribution_of_{col}.jpg'
    plt.savefig(filename, dpi=300, bbox_inches='tight')

    plt.show()

## Function 2: qn_stats
- a function to handle quantitative data 
- displays a general summary for the column parameter
- skewness: tells us the shape and spread of the data from the center
- kurtosis: tells us how much data is far from the center
- plots histogram + kde
- runs normality test to see if the data is normally distributive
- plots QQ-plot to visualize how much the data follows the normal distribution

In [ ]:
def qn_stats(df, col):
    """
    Prints out summary for quantitative columns and creates 
    histogram + KDE plots.

    Parameters:
    df (DataFrame): The DataFrame containing the data.
    col (str): The name of the numeric column to summarize.
    """
    print(f"\n--- Numerical Summary: {col} ---")
    desc = df[col].describe()
    print(desc)

    # Boxplot
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot of {col}")
    plt.xlabel(col)
    filename = f"Boxplot_of_{col}.jpg"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Mode: {df[col].mode()[0]}")
    print(f"Skewness: {skew(df[col].dropna()):.2f}")
    print(f"Kurtosis: {kurtosis(df[col].dropna()):.2f}")
    
    # Histogram + KDE
    sns.histplot(df[col], kde=True, bins=20)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel('Frequency')
    filename = f"Distribution_of_{col}.jpg"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

    # Normality Test
    stat, p=normaltest(df[col])
    print(f"\nD'Agostino and Pearson Test:")
    print(f" Statistic = {stat:.4f}, p-value = {p:.4f}")
    if p < 0.05:
        print("Data is not normally distributed.")
    else:
        print("Data is normally distributed.")

    # QQ Plot
    plt.figure(figsize=(6,6))
    probplot(df[col], dist='norm', plot=plt)
    plt.title(f'QQ-Plot of {col}')
    plt.xlabel("Theoretical Quantiles")
    plt.ylabel('Sample Quantiles')
    plt.grid(True)
    filename = f'QQ-Plot_of_{col}.jpg'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()


## Function 3: dt_stats
- properly handles datetime data type variables
- summarizes all the dates from the datetime column
- visualizes monthly and yearly time series plots

In [ ]:
def dt_stats(df, col):
    """
    Summarizes datetime columns and plots time series trends.
    
    Parameters:
    df (DataFrame): The DataFrame containing the data.
    col (str): The name of the numeric column to summarize.
    """
    print(f"\n--- Datetime Summary: {col} ---")
    print(f"Min date: {df[col].min()}")
    print(f"Max date: {df[col].max()}")
    print(f"Range: {df[col].max() - df[col].min()}")
    print(f"Median: {df[col].median()}")
    print(f"Mode: {df[col].mode()[0]}")
    print(f"Unique dates: {df[col].nunique(dropna=False)}")

    #Counts per year
    year_counts = df[col].dt.year.value_counts().sort_index()
    print("\nCounts per year:")
    print(year_counts)

    # Yearly counts
    dt_yearly = df.set_index(col).resample('Y').size()
    plt.figure(figsize=(14, 7))
    dt_yearly.plot(marker='o')
    plt.title(f"Yearly Count of {col}")
    plt.xlabel("Year")
    plt.ylabel("Count")
    plt.grid(True)
    plt.tight_layout()
    filename=f'Yearly_Count_of_{col}.jpg'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()

## Using a for loop to go through all the columns

In [ ]:
# Drop Employee ID from the columns
df = df.drop(columns=['EmployeeID'])

# Temporarily turns Attrition and OverTime variables as categories
df['Attrition'] = df['Attrition'].astype('category')
df['OverTime'] = df['OverTime'].astype('category')

# Loop through all columns and run stats, tests, and plots
for col in df.columns:
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        dt_stats(df, col)
    elif pd.api.types.is_numeric_dtype(df[col]):
        qn_stats(df, col)
    else:
        ql_stats(df, col)

## Key Observations:
### Gender
- 46.07% of employees identify as female
- 44.24% of employees identify as male
- 8.78% of employees identify as non-binary
- 0.91% of employees did not state their gender identity 
- Slightly more employees identify as female than male, with a small proportion identifying was non-binary or not disclosing

### Age
- The average age of employees is 30.78 years (SD = 7.93)
- The youngest employee is 18 years old, and the oldest is 51 years old.
- 25% of employees are under 25 years old, 50% are under 28 years old, and 75% are under 37.
- The most common age (mode) is 25 years old. 
- The distribution of ages is right-skewed (skewness = 0.67), indicating a higher concentration of younger employees.
- The kurtosis (-0.72) indicates a platykurtic distribution with a flatter shape and lighter tails. 
- THe histogram shows a clear peak at age 25 and a more even spread of ages after 30.
- Since the p-value is 0 (and is less than 0.05) we reject the null hypothesis and say that the distribution of age is not normal. 
- The QQ-Plot shows deviations from the theoretical normal line 

### BusinessTravel
- 70.65% of employees engage in some travel which is the most common.
- 20.24% of employees are frequent travelers.
- 9.11% of employees do not travel at all.
- Overall, more than half of the employees have some travel requirements as part of their role.

### Department
- 63.45% of the company work in the Technology department
- 32.03% of the company work in the Sales department
- 4.52% of the company work in the Human resources department
- Overall, more than half of the employees work in Technology.

### DistranceFromHome (KM)
- The average distance distance from home for an employee is 22.3 km.
- The standard deviation is 12.90 km.
- The shortest commute is 1 km, while the longest is 45 km.
- 25% of the company lives within 12 km away from work, 50% within 22 km, and 75% within 33 km.
- The most common commute distance is 14 km. 
- The skewness (0.07) is close to 0 which indicates a nearly symmetrical distribution.
- The kurtosis (-1.16) suggests a platykurtic distribution with lighter tails and fewer extreme values.
- The distribution plot shows a very flat peak with no extreme outliers.
- Overall, employee commute distances vary widely but are symmetrical distributed without strong outliers.
- Since the p-value is 0 (and is less than the significance level of 0.05), we reject the null hypothesis and say that the distribution of distance from home is not normal.
- The QQ-plot confirms the distribution deviates from a normal distribution.

### State
- 60.35% of employees live in California.
- 27.57% of employees live in New York.
- 12.07% of employees live in Illinois. 
- The majority of employees residing in California suggests that the company is likely based there, with smaller offices in New York and Illinois.

### Ethnicity
- 49.61% of employees identify as white.
- 16.38% of employees identify as mixed.
- 15.96% of employees identify as black.
- 10.18% of employees identify as asian.
- 4.23% of employees identify as American Indian or Alaska Native.
- 2.34% of employees identify as Native Hawaiian or Other Pacific Islander.
- 1.3% of employees identify as Other.
- Just under half of the company identifies as White.


### Education
- 39.26% of employees have a Bachelor's degree.
- 24.94% of employees have a Master's degree.
- 20.14% of employees have a GED (General Educational Development certificate).
- 12.52% of employees have no formal qualifications.
- 3.15% of employees have a Doctorate degree.
- The company hires mostly employees with secondary or higher education, with Bachelor's and Master's degrees being the most common.
- However, since more employees hold a GED or no formal qualifcications than those with a Doctorate, advanced degrees like Doctorates may not be a significant factor in hiring decisions.

### Education Field
- 29.71% of employees majored in Computer Science.
- 23% of employees majored in Information Systems.
- 22.98% of employees majored in Marketing.
- 6.78% of employees majored in Business Studies.
- 6.65% of employees majored in Economics.
- 5.50% of employees majored in other studies.
- 3.4% of employees obtained a technical degree.
- 1.98% of employees majored in human resources. 
- It would be interesting to compare these Education Fields with the Department column to see how closely aligned employees' majors are to their departments.
- Analyzing Education Field against attrition might be less informative, since the majority of employees come from just a few majors (Computer Science, Information Systems, and Marketing).

### Job Role
- The 3 job roles with the most employees (from largest to smallest) are: sales executive (22.67%), data scientist (20.27%), and software engineer (19.88%).
- There are 10 additional job roles each comprising less than 9% of employees.
- This distribution aligns well with the most common education fields - Computer Science, Information Systems, and Marketing
- This suggests the company may prioritize hiring candidates whose educational abckgrounds closely match their job roles, leading to a strong alignment between qualifications and responsibilites.

### Marital Status
- 42.67% of employees are married
- 37.83% of employees are single
- 19.50% of employees are divorced
- The number of married and single employees is fairly close, with only a difference of 324 employees.
- Given that 50% of employees are under 28 and 75% are under 38, it makes sense that many are either single or newly married, reflecting typical young adult life stages.


### Salary
- The average annual salary of employees is $111,061.75.
- The standard deviation is $98,268.10, indictating high variability in salries.
- The lowest salary recorded is $20,387, while the highest is $547,204 per year.
- The 25th percentile salary is $45,276, the median (50th percentile) is $75,667, and the 75th percentile is $127,427.
- The boxplot shows most salaries are concentrated on the lower end, with a significant number of outliers on the higher end, indicating less employees earn substantially more than the majority.
- The most common salary (mode) is $107,863.
- Positive skewness (1.91) indicates the distribution is right-tailed, with more extreme high values.
- The kurtosis is 3.79, suggesting a distribution with a sharper peak and heavier tails than a normal distribtion.
- Since the p-value is 0 (less than the significance level of 0.05), we reject the null hypothesis and say that it is not normally distributed.
- The QQ plot supports this, which shows deviations from the expected quantiles of a normal distribution.


### Stock Option Level
- 47.74% of employees were granted 0 stocks.
- 37.50% of employees were granted 1 stock. 
- 9.57% of employees were granted 2 stocks.
- 5.19% of employees were granted 3 stocks. 
- This suggests that the company does not heavily rely on stock grants as part of employee compensation.

### Overtime
- 66.66% of employees do not work overtime.
- 33.34% of employees do work overtime. 
- It could be interesting to analyze whether working overtime impacts employee attrition.

### Hire Date
- The first day of hiring was January 3rd, 2012.
- The most recent day of hiring was December 3rd, 2022.
- The time span between the first and most recent hire is $3,987 days.
- The median hire date is March 28th, 2015.
- The date with the most hires was April 23rd, 2012.
- There are 1,048 days where only one employee was hired.
- The years with the highest number of hires, from highest to lowest, are 2012, 2013, and 2014.
- The year with the fewest hires was 2022.
- The time series plot shows a general decline in hiring over the years, with a small hearing spike in 2018.
- We can infer that many employees hired between 2012-2014 are still employed, resulting in fewer open roles in recent years.


### Attrition
- 66.3% of employees stayed with the company upon hire.
- 33.7% of employees eventually left the company upon hire. 
- Considering the decrease in hiring over recent years, the retention rate aligns with the trend of fewer new hires.


### Years At Company
- The average tenure of employees at this company is approximately 6 years.
- The standard deviation of tenure is 3.33 years.
- The shortest tenure recorded is 0 years.
- The longest tenure recorded is 10 years.
- Given the large hiring surge in 2012, it's expected to see many employees around the 10-year mark.
- 25% of employees have stayed for 3 years or less.
- 50% of employees stayed for 6 years or less.
- 75% of employees statyed for 9 years or less.
- The boxplot reveals a longer left tail than right, indicating mroe employees with shorter tenures.
- The most common tenure (mode) is 10 years.
- Negative skewness (-0.32) indicates the distribution is left-skewed.
- The kurtosis below 3 suggests a flatter distribution with lighter tails, meaning fewer extreme tenure values.
- The distribution plot shows a relatively consistent number of employees with tenures between 0 to 7 years.
- There is an increase in employees with tenure around 8 years and above, likely reflecting the 2012 hiring surge.
- Since the p-value is 0 (and less than the significance level of 0.05), we reject the null hypothesis and conclude the tenure data is not normally distributed.
- The QQ-Plot confirms this by showing flatter tails drifting away from the theoretical line on both ends.


### Years In Most Recent Role
--- Numerical Summary: YearsInMostRecentRole ---
count    6709.000000
mean        2.852884
std         2.813943
min         0.000000
25%         0.000000
50%         2.000000
75%         5.000000
max        10.000000
- The average number of years an employee has stayed in their recent role is 2.86.
- The average number of years an employee has stayed in their recent role has a standard deviation of 2.81.
- The lowest number of years an employee has been in their recent role is 0. 
- The highest number of years an employee has been in their recent role is 10.
- 25% of employees have been in their recent role for 0 years.
- 50% of employees have been in their recent role for 2 years.
- 75% of employees have been in their recent role for 5 years. 
- The boxplot has a very long right tail, which means the amount of employees who have stayed in their recent role for longer is less. 

Mode: 0
Skewness: 0.78
Kurtosis: -0.48
- The most common amount of years an employee has been in their recent role is 0. 
- Have most employees recently been promoted?
- Seeing from the plot that the amount of employees that are still in the same role for 1 year - 10 years are decreasing, we can guess that the employees who have been with this company for longer, have recently changed their role.
- The skewness is positive so the data is right-skewed and is right-tailed.
- The kurtosis is less than 3, which means the data has a flatter peak and has lighter tails.

D'Agostino and Pearson Test:
 Statistic = 651.5986, p-value = 0.0000
Data is not normally distributed.
- Since the p-value is 0 and is less than the significant level (alpha=0.05), we reject the null hypothesis and say that it is not normally distributed.
- The QQ-plot shows that there are a lot of negative quantiles

### Years Since Last Promotion
--- Numerical Summary: YearsSinceLastPromotion ---
count    6709.000000
mean        4.254136
std         3.179384
min         0.000000
25%         1.000000
50%         4.000000
75%         7.000000
max        10.000000
- The average amount of years an employee was last promoted is 4.
- The average amount of years an employee was last promoted has a standard deviation of 3.12 years.
- The lowest amount of years an employee has last been promoted is 0.
- The highest amount of years an employee has last been promoted is 10.
- 25% of employees were last promoted a year ago. 
- 50% of employees were last promoted 4 years ago.
- 75% of employees were last promoted 7 years ago.
- The boxplot shows a slightly longer right whisker.

Mode: 0
Skewness: 0.18
Kurtosis: -1.21
- The most common amount of years for employees since their last promotion is 0. 
- The skewness is close to 0, which means the data is near symmmetrical on both sides of the mean.
- The kurtosis is less than 3 so the data has a flatter peak and has light tails. 
- The plot shows that the amount of years since an employee has last been promoted stays about the same from 1-10 years. 
- It also shows that most of the data is crowded near the left.

D'Agostino and Pearson Test:
 Statistic = 6121.6609, p-value = 0.0000
Data is not normally distributed.
- Since the p-value is 0 and is less than the significance level (alpha=0.05), we reject the null hypothesis and say that it is not normally distributed.
- The QQ-plot also shows us that the quantiles do not follow the normal distribution.

### Years With Current Manager
--- Numerical Summary: YearsWithCurrManager ---
count    6709.000000
mean        2.814577
std         2.796340
min         0.000000
25%         0.000000
50%         2.000000
75%         5.000000
max        10.000000
- The average amount of years an employee has been with their current manager is 2.81 years.
- The amount of years an employee has been with their current manager has a standard deviation of 2.8 years.
- The lowest amount of years an employee has been with their current manager is 0.
- The highest amount of years an employee has been with their current manager is 10.
- 25% of employees have stayed with their current manager for 0 years.
- 50% of employees have stayed with their current manager for 2 years.
- 75% of employees have stayed with their current manager for 5 years.
- The boxplot shows that most of the data is crowded to the left with a long right whisker.


Mode: 0
Skewness: 0.84
Kurtosis: -0.36
- The skewness is positive which means the data is right skewed.
- The kurtosis is less than 3 which means the peak is flatter and there are less extreme outliers in the data.
- The graph shows the largest amount of employees have stayed with their current manager for 0 years. 
- The mode being 0 confirms this. 
- We also see that most employees have not stayed with their current manager for longer than a year. 
- However, we know that a lot of employees were just promoted under a year ago, so while there are less employees who have stayed with the current manager for 1+ years, it's less likely that quick promotions is a characteristic for this company.

D'Agostino and Pearson Test:
 Statistic = 668.3431, p-value = 0.0000
Data is not normally distributed.
- Since the p-value is 0 and is less than the significance level (alpha=0.05), we reject hte null hypothesis and say that the data is not normally distributed.
- The QQ plot also confirms this.

### Yearly Count of ReviewDate
--- Datetime Summary: ReviewDate ---
Min date: 2013-01-02 00:00:00
Max date: 2022-12-31 00:00:00
Range: 3650 days 00:00:00
Median: 2019-09-15 00:00:00
Mode: 2022-05-22 00:00:00
Unique dates: 2771

Counts per year:
2013     151
2014     285
2015     411
2016     529
2017     630
2018     722
2019     836
2020     965
2021    1055
2022    1125
- The oldest review date was conducted on January 2nd, 2013.
- The most recent review date was conducted on December 31st, 2022.
- The amount of days between the oldest review and the most recent review is 3650 days.
- The review date that is in the middle is September 15th, 2019.
- The day with the most reviews was on May 22nd, 2022.
- We can suggest this is when most of the employees were promoted.
- There are 2771 days where there was only one review done. 
- The year with the most review dates is 2022.
- Since we know most employees were just promoted less than a year ago, we can say that most were promoted in 2022, rather than demoted.
- The time series plot shows a steady increase, almost a linear rate between each year.
- This may be why most employees have not left the company yet.
- Perhaps when the huge hiring batch between 2012-2014 retires or leaves, there will be another surge of new hires to replace them.

### Environment Satisfaction
- 32.96% of employees rate their environment a 3.
- 32.42% of employees rate their environment a 4.
- 30.50% of employees rate their environment a 5.
- 2.10% of employees rate their environment a 2.
- 2.03% of employees rate their environment a 1.
- There seems to be a similar amount of employees who rate their environment between 3, 4, 5.

### Job Satisfaction
--- Categorical Summary: JobSatisfaction ---
   Count  Percentages
4   1685        25.12
2   1674        24.95
3   1651        24.61
5   1569        23.39
1    130         1.94
Unique categories: 5
Most frequent: 4
- 25.12% of employees rate their job a 4.
- 24.95% of employees rate their job a 2.
- 24.61% of employees rate their job a 3.
- 23.39% of employees rate their job a 5
- 1.94% of employees rate their job a 1. 
- There seems to be a similar amount of employees who rated their job a 3, 4, or 5. 

--- Categorical Summary: RelationshipSatisfaction ---
   Count  Percentages
4   1690        25.19
2   1687        25.15
3   1626        24.24
5   1569        23.39
1    137         2.04
Unique categories: 5
Most frequent: 4
- 25.19% of employees feel satisfied with their relationships at work.
- 25.15% of employees feel not satisfied with their relationships at work.
- 24.24% of employees feel neutral with their relationships at work.
- 23.39% of employees feel very satisfied with their relationships at work.
- 2.04% of employees feel very satisfied with their relationships at work.
- There seems to be a similar amount of employees who rated their satisfaction with relationships between a rating of 2-5.

### Training Opportunities
--- Categorical Summary: TrainingOpportunitiesWithinYear ---
   Count  Percentages
3   2301         34.3
1   2214         33.0
2   2194         32.7
Unique categories: 3
Most frequent: 3
- 34.3% had 3 training opportunities.
- 33% had 1 training opportunity. 
- 32.7% had 2 training opportunities.

### Training Opportunities
--- Categorical Summary: TrainingOpportunitiesTaken ---
   Count  Percentages
1   2449        36.50
0   2372        35.36
2   1288        19.20
3    600         8.94
Unique categories: 4
Most frequent: 1
- 36.50% of employees took 1 training opportunty.
- 35.36% of employees took 0 training opportunities.
- 19.20% of employees took 2 training opportunities.
- 8.94% of employees took 3 traiing opportunities. 
- Most employees either took 0 or 1 training opportunity. 

### Work Life Balance
--- Categorical Summary: WorkLifeBalance ---
   Count  Percentages
4   1706        25.43
2   1702        25.37
3   1670        24.89
5   1510        22.51
1    121         1.80
Unique categories: 5
Most frequent: 4
- 25.43% of employees feel satisfied with their work life balance.
- 25.37% of employees feel not satisfied with their work life balance.
- 24.89% of employees feel neutral with their work life balance.
- 22.51% of employees feel very satisfied with their work life balance.
- 1.80% of employees feel very not satisfied with their work life balance.
- Similarities for ratings 2-5.

### Self Rating
--- Categorical Summary: SelfRating ---
   Count  Percentages
3   2290        34.13
4   2236        33.33
5   2183        32.54
1      0         0.00
2      0         0.00
Unique categories: 3
Most frequent: 3
- 34.13% of employees feel neutral with themselves.
- 33.33% of employees feel satisfied with themselves.
- 32.54% of employees feel very satisfied with themselves. 
- 0% of employees feel dissatisfied with themselves.
- 0% of employees feel very dissatisfied with themselves.

--- Categorical Summary: ManagerRating ---
   Count  Percentages
3   2223        33.13
4   2220        33.09
2   1192        17.77
5   1074        16.01
1      0         0.00
Unique categories: 4
Most frequent: 3
- 33.13% of employees were rated a 3 by their manager.
- 33.09% of employees were rated a 4 by their manager.
- 17.77% of employees were rated a 2 by their manager.
- 16.01% of employees were rated a 5 by their manager.
- 0.00% of employees were rated a 1 by their manager.
- There are similar amounts of employees who were rated a 3 or 4 by their manager.






















